In [2]:
try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()

Output()

╭─ kiara plugin(s) installed ─────────────────────────────────────────────────────────────────────────────────────╮
│ Installed packages:                                                                                             │
│                                                                                                                 │
│  • kiara-plugin.onboarding                                                                                      │
│  • kiara-plugin.tabular                                                                                         │
│                                                                                                                 │
│ Depending on the state of your current environment, you might see error messages below, in which case you'll    │
│ have to restart the jupyter kernel manually.                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [3]:
from kiara import KiaraAPI

# create the default api instance
api = KiaraAPI.instance()


In [4]:
# we can use any of the functions of the api instance
# documentation [here](https://dharpa.org/kiara/latest/reference/kiara/interfaces/python_api/__init__/#kiara.interfaces.python_api.KiaraAPI)

api.list_operation_ids()

['create.database.from.file',
 'create.database.from.file_bundle',
 'create.database.from.table',
 'create.table.from.file',
 'create.table.from.file_bundle',
 'date.check_range',
 'date.extract_from_string',
 'download.file',
 'download.file_bundle',
 'export.file.as.file',
 'export.table.as.csv_file',
 'extract.date_array.from.table',
 'file_bundle.pick.file',
 'file_bundle.pick.sub_folder',
 'import.database.from.local_file_path',
 'import.file',
 'import.file_bundle',
 'import.table.from.local_file_path',
 'import.table.from.local_folder_path',
 'jupyter.example',
 'list.contains',
 'logic.and',
 'logic.nand',
 'logic.nor',
 'logic.not',
 'logic.or',
 'logic.xor',
 'onboard.zenodo_record',
 'parse.date_array',
 'query.database',
 'query.table',
 'string_filter.tokens',
 'table.pick.column',
 'table_filter.drop_columns',
 'table_filter.select_columns',
 'table_filter.select_rows']

In [5]:
# most likely, we'll want to run an operation, let's pick one

op_id = "download.file"

# now, let's check information about the operation, esp. what input(s) it requires
api.retrieve_operation_info(op_id)


Documentation                                                                                                                              
                     -- n/a --                                                                                                              
                                                                                                                                            
 Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         onboarding                                                                                                
                     Labels       package: kiara_plugin.onboarding                                                                          
                     References   source_repo: ]8;id=297426;https://github.com/DHARPA-Project/kiara_plugin.onboarding\https://github.com/DHARPA-Project/kiara_plugin.onboarding]8;;\                                    
                                  documentation: ]8;id=714656;https://DHARPA-Project.github.io/kiara_plugin.onboarding/\https://DHARPA-Project.github.io/kiara_plugin.onboarding/]8;;\                                  
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   -- n/a --                                                                                              
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type     description                                    Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       url          string   The url of the file to download.               yes        -- no default --     
                                       file_name    string   The file name to use for the downloaded        no         -- no default --     
                                                             file.                                                                          
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name          type   description                                                               
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       file                file   The downloaded file.                                                      
                                  

In [6]:
# ok, we need an 'url' value, and an (optional) file_name:

inputs = {
    "url": "https://raw.githubusercontent.com/DHARPA-Project/kiara.examples/main/examples/data/journals/JournalNodes1902.csv",
    "file_name": "JournalNodes1902.csv"
}

# now, run it
results = api.run_job(op_id, inputs=inputs)
# ...and print a result preview
results

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field               value                                                                                                              │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   download_metadata                                                                                                                      │
│                         dict data     {                                                                                                  │
│                                         "response_headers": [                                                                            │
│                                           {                                                                                              │
│                                             "connection": "keep-alive",                                                                  │
│                                             "content-length": "7436",                                                                    │
│                                             "cache-control": "max-age=300",                                                              │
│                                             "content-security-policy": "default-src 'none'; style-src 'unsafe-inline'; sandbox",         │
│                                             "content-type": "text/plain; charset=utf-8",                                                 │
│                                             "etag": "W/\"641ae85d69e5836d27ea8906aba0a33b48b0f3ed0ed4c40d21a07fccebdd238d\"",            │
│                                             "strict-transport-security": "max-age=31536000",                                             │
│                                             "x-content-type-options": "nosniff",                                                         │
│                                             "x-frame-options": "deny",                                                                   │
│                                             "x-xss-protection": "1; mode=block",                                                         │
│                                             "x-github-request-id": "B0DE:75FF:434241:5ADF15:63CBF8F0",                                   │
│                                             "content-encoding": "gzip",                                                                  │
│                                             "accept-ranges": "bytes",                                                                    │
│                                             "date": "Sat, 21 Jan 2023 14:38:40 GMT",                                                     │
│                                             "via": "1.1 varnish",                                                                        │
│                                             "x-served-by": "cache-hhn-etou8220049-HHN",                                                  │
│                                             "x-cache": "MISS",                                                                           │
│                                             "x-cache-hits": "0",                                                                         │
│                                             "x-timer": "S1674311921.820972,VS0,VE162",                                                   │
│                                             "vary": "Authorization,Accept-Encoding,Origin",                                              │
│                                             "acces

In [7]:
# this looks good, we are really only interested in the 'file' output, let's store it in a variable for later:

downloaded_file = results["file"]

# now we want to do something with that file, specifically, convert it into a table
# the operation id for this is:

op_id = "create.table.from.file"

# and let us check information about the operation, esp. its input(s)
api.retrieve_operation_info(op_id)


Documentation                                                                                                                              
                     Create a table from a file, trying to auto-determine the format of said file.                                          
                                                                                                                                            
 Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=151365;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=653162;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Create a table from a file, trying to auto-determine the format of said file.                          
                                                                                                                                            
                     Inputs                                                                                                                 
                                       field name   type   description                                      Required   Default              
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       file         file   The source value (of type 'file').               yes        -- no default --     
                                                                                                                                            
                                                                                                                                            
                     Outputs                                                                                                                
                                       field name   type    description                                                                     
                                      ──────────────────────────────────────────────────────────────────────────────────────────────────    
                                       table        table   The result value (of type 'table').                                             
                                                                                                                                            
                  

In [8]:
# same procedure as before, we assemble the input(s), and run a kiara job

inputs = {
    "file": downloaded_file
}

# now, run it
results = api.run_job(op_id, inputs=inputs)
# ...and print a result preview
results


╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   table                                                                                                                                  │
│             Id    Label            JournalType      City        CountryNetworkT   PresentDayCoun   Latitude    Longitude    Language     │
│            ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│             75    Psychiatrische   specialized: p   Amsterdam   Netherlands       Netherlands      52.366667   4.9          Dutch        │
│             36    The American J   specialized: p   Baltimore   United States     United States    39.289444   -76.615278   English      │
│             208   The American J   specialized: p   Baltimore   United States     United States    39.289444   -76.615278   English      │
│             295   Die Krankenpfl   specialized: t   Berlin      German Empire     Germany          52.52       13.405       German       │
│             296   Die deutsche K   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             300   Therapeutische   specialized: t   Berlin      German Empire     Germany          52.52       13.405       German       │
│             1     Allgemeine Zei   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             7     Archiv für Psy   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             10    Berliner klini   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             13    Charité Annale   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             21    Monatsschrift    specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             29    Virchows Archi   specialized: a   Berlin      German Empire     Germany          52.52       13.405       German       │
│             31    Zeitschrift fü   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             42    Vierteljahrssc   specialized: a   Berlin      German Empire     Germany          52.52       13.405       German       │
│             47    Centralblatt f   specialized: p   Berlin      German Empire     Germany          52.52       13.405       German       │
│             50    Russische medi   general medici   Berlin      German Empire     Germany          52.52       13.405       German       │
│             ...   ...              ...              ...         ...               ...              ...         ...          ...          │
│             ...   ...              ...              ...         ...               ...              ...         ...          ...          │
│             277   L'arte medica    general medici   Turin       Italy             Italy            45.079167   7.676111     Italian      │
│             288   Allgemeine öst   specialized: a   Vienna      Austro-Hungaria   Austria          48.2        16.366667    German       │
│             18    Jahrbücher für   specialized: p   Vienna      Austro-Hungaria   Austria          48.2        16.366667    German       │
│             30    Wiener klinisc   general medic

In [9]:
# we are really only interested in the 'table' output field, let's get the 'Value' object for it

table_val = results["table"]
table_val  # this displays some general (mostly internal) information and metadata for the value

 value_id            a0de70ac-3a94-48d4-ad88-1957589843df                                                                                  
  kiara_id            76b2fda2-bbef-451c-aebf-3bf7a0e3e707                                                                                  
                                                                                                                                            
                      ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  
  data_type_info                                                                                                                            
                        data_type_name     table                                                                                            
                        data_type_config   {}                                                                                               
                        characteristics    {                                                                                                
                                             "is_scalar": false,                                                                            
                                             "is_json_serializable": false                                                                  
                                           }                                                                                                
                        data_type_class                                                                                                     
                                             python_class_name    TableType                                                                 
                                             python_module_name   kiara_plugin.tabular.data_types.table                                     
                                             full_name            kiara_plugin.tabular.data_types.table.TableType                           
                                                                                                                                            
                                                                                                                                            
  destiny_backlinks   {}                                                                                                                    
  enviroments         None                                                                                                                  
  property_links      {                                                                                                                     
                        "metadata.python_class": "c89a4f8f-973e-4163-be75-9c6a5974ea27",                                                    
                        "metadata.table": "48ecf1ac-0371-4172-8eb1-12184a6111bd"                                                            
                      }                                                                                                                     
  value_hash          zdpuAn89Et1ENzfoASJRYcWEceyfRiPg664mN4nnHLFnjRLyg                                                                     
  value_schema                                                                                                                              
                        type          table                                                                                                 
                        type_config   {}                                                                                                    
                        default       __not_set__                                                                                           
                        optional      False          

In [10]:
# maybe we want to run a sql query against the table, let's find a good operation

api.list_operation_ids("query")

['query.database', 'query.table']

In [11]:
# 'query.table' looks good, let's see...

api.retrieve_operation_info('query.table')

Documentation                                                                                                                              
                     Execute a sql query against an (Arrow) table.                                                                          
                                                                                                                                            
                     The default relation name for the sql query is 'data', but can be modified by the 'relation_name' config               
                     option/input.                                                                                                          
                                                                                                                                            
                     If the 'query' module config option is not set, users can provide their own query, otherwise the pre-set one will be   
                     used.                                                                                                                  
                                                                                                                                            
 Author(s)                                                                                                                                  
                     Markus Binsteiner   markus@frkl.io                                                                                     
                                                                                                                                            
 Context                                                                                                                                    
                     Tags         tabular                                                                                                   
                     Labels       package: kiara_plugin.tabular                                                                             
                     References   source_repo: ]8;id=795634;https://github.com/DHARPA-Project/kiara_plugin.tabular\https://github.com/DHARPA-Project/kiara_plugin.tabular]8;;\                                       
                                  documentation: ]8;id=207497;https://DHARPA-Project.github.io/kiara_plugin.tabular/\https://DHARPA-Project.github.io/kiara_plugin.tabular/]8;;\                                     
                                                                                                                                            
 Operation details                                                                                                                          
                     Documentation   Execute a sql query against an (Arrow) table.                                                          
                                                                                                                                            
                                     The default relation name for the sql query is 'data', but can be modified by the 'relation_name'      
                                     config option/input.                                                                                   
                                                                                                                                            
                                     If the 'query' module config option is not set, users can provide their own query, otherwise the       
                                     pre-set                                                                                                
                                     one will be used.                                                                                      
                                        

In [12]:
# ok, given this information, we only need to provide the 'table' and 'query', and reference the string data in the sql...

inputs = {
    "table": table_val,
    "query": "SELECT Label, City from data where City like 'Berlin'"
}
results = api.run_job("query.table", inputs=inputs)
results

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field          value                                                                                                                   │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────                      │
│   query_result                                                                                                                           │
│                    Label                                                                                  City                           │
│                   ───────────────────────────────────────────────────────────────────────────────────────────────                        │
│                    Die Krankenpflege                                                                      Berlin                         │
│                    Die deutsche Klinik am Eingange des zwanzigsten Jahrhunderts                           Berlin                         │
│                    Therapeutische Monatshefte                                                             Berlin                         │
│                    Allgemeine Zeitschrift für Psychiatrie                                                 Berlin                         │
│                    Archiv für Psychiatrie und Nervenkrankheiten                                           Berlin                         │
│                    Berliner klinische Wochenschrift                                                       Berlin                         │
│                    Charité Annalen                                                                        Berlin                         │
│                    Monatsschrift für Psychiatrie und Neurologie                                           Berlin                         │
│                    Virchows Archiv                                                                        Berlin                         │
│                    Zeitschrift für pädagogische Psychologie und Pathologie                                Berlin                         │
│                    Vierteljahrsschrift für gerichtliche Medizin und öffentliches Sanitätswesen            Berlin                         │
│                    Centralblatt für Nervenheilkunde und Psychiatrie                                       Berlin                         │
│                    Russische medicinische Rundschau                                                       Berlin                         │
│                    Deutsche Aerzte-Zeitung                                                                Berlin                         │
│                    Monatsschrift für Geburtshülfe und Gynäkologie                                         Berlin                         │
│                    Archiv für klinische Chirurgie                                                         Berlin                         │
│                    Zeitschrift für klinische Medicin                                                      Berlin                         │
│                    Deutsche militärärztliche Zeitschrift                                                  Berlin                         │
│                    Jahresbericht über die Leistungen und Fortschritte auf dem Gebiete der Neurologi ...   Berlin                         │
│                    Ärztliche Sachverständigen-Zeitung                                                     Berlin                         │
│                    Zeitschrift für die Behandlung Schwachsinniger und Epileptischer                       Berlin                         │
│                    Der Pfarrbote                  

In [13]:
# let's get the value of the 'query_result' field...

query_result = results['query_result']

# and have a look at its lineage (even though it's not particularly interesting yet)
query_result.lineage


query.table
├── input: query (string) = d257cbe6-553f-4228-956c-3174df7e47d4
├── input: relation_name (string) = b57df7d2-a972-4801-a9b7-12324db9db65
└── input: table (table) = a0de70ac-3a94-48d4-ad88-1957589843df
    └── create.table
        └── input: file (file) = 3839ac96-70c8-40c4-a329-b6323ff798b1
            └── download.file
                ├── input: file_name (string) = 01430591-571c-4798-bc32-f8479bf7d33b
                └── input: url (string) = 29ba163f-02da-4243-bc30-910264674357

In [14]:
# and some (type specific) properties...

query_result.property_values


╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field                   value                                                                                                          │
│  ─────────────────────────────────────────────────────────────────────────────────────────                                               │
│   metadata.python_class   {                                                                                                              │
│                             "python_class": {                                                                                            │
│                               "python_class_name": "KiaraTable",                                                                         │
│                               "python_module_name": "kiara_plugin.tabular.models.table",                                                 │
│                               "full_name": "kiara_plugin.tabular.models.table.KiaraTable"                                                │
│                             }                                                                                                            │
│                           }                                                                                                              │
│   metadata.table          {                                                                                                              │
│                             "table": {                                                                                                   │
│                               "column_names": [                                                                                          │
│                                 "Label",                                                                                                 │
│                                 "City"                                                                                                   │
│                               ],                                                                                                         │
│                               "column_schema": {                                                                                         │
│                                 "Label": {                                                                                               │
│                                   "type_name": "string",                                                                                 │
│                                   "metadata": {                                                                                          │
│                                     "arrow_type_id": 13                                                                                  │
│                                   }                                                                                                      │
│                                 },                                                                                                       │
│                                 "City": {                                                                                                │
│                                   "type_name": "string",                                                                                 │
│                                   "metadata": {                                                                                          │
│                                     "arrow_type_id": 13                                                                                  │
│                                   }               

In [15]:
# the value is of type 'table', we can access the data as a Python object

query_table_data = query_result.data

# we can check out what (Python) type is used under the hood
type(query_table_data)

kiara_plugin.tabular.models.table.KiaraTable

In [16]:
# and if we look up documentation about this type, we can use some of its
# utility methods etc. For this type, the documentation is here: https://dharpa.org/kiara_plugin.tabular/latest/reference/kiara_plugin/tabular/models/table/#kiara_plugin.tabular.models.table.KiaraTable
#
# TODO: implement discoverability for these types of things, so links to documentation can be accessd
# directly via the Python object

# for example, we can get the table as a Pandas dataframe:

df = query_table_data.to_pandas()
df


,Label,City
0,Die Krankenpflege,Berlin
1,Die deutsche Klinik am Eingange des zwanzigste...,Berlin
2,Therapeutische Monatshefte,Berlin
3,Allgemeine Zeitschrift für Psychiatrie,Berlin
4,Archiv für Psychiatrie und Nervenkrankheiten,Berlin
5,Berliner klinische Wochenschrift,Berlin
6,Charité Annalen,Berlin
7,Monatsschrift für Psychiatrie und Neurologie,Berlin
8,Virchows Archiv,Berlin
9,Zeitschrift für pädagogische Psychologie und P...,Berlin
